# install  

* pip install transitions

In [9]:
class Matter(object):
    pass

lump = Matter()

from transitions import Machine

# The states
states=['solid', 'liquid', 'gas', 'plasma']

# And some transitions between states. We're lazy, so we'll leave out
# the inverse phase transitions (freezing, condensation, etc.).
transitions = [
    { 'trigger': 'melt', 'source': 'solid', 'dest': 'liquid' },
    { 'trigger': 'evaporate', 'source': 'liquid', 'dest': 'gas' },
    { 'trigger': 'sublimate', 'source': 'solid', 'dest': 'gas' },
    { 'trigger': 'ionize', 'source': 'gas', 'dest': 'plasma' }
]

# Initialize
machine = Machine(lump, states=states, transitions=transitions, initial='liquid')



In [10]:
lump.evaporate()

True

In [13]:
# Our old Matter class, now with  a couple of new methods we
# can trigger when entering or exit states.
from transitions import Machine, State

class Matter(object):
    def say_hello(self): print("hello, new state!")
    def say_goodbye(self): print("goodbye, old state!")

lump = Matter()

# Same states as above, but now we give StateA an exit callback
states = [
    State(name='solid', on_exit=['say_goodbye']),
    'liquid',
    { 'name': 'gas', 'on_exit': ['say_goodbye']}
    ]

machine = Machine(lump, states=states)
machine.add_transition('sublimate', 'solid', 'gas')

# Callbacks can also be added after initialization using
# the dynamically added on_enter_ and on_exit_ methods.
# Note that the initial call to add the callback is made
# on the Machine and not on the model.
machine.on_enter_gas('say_hello')

# Test out the callbacks...
machine.set_state('solid')
lump.sublimate()

goodbye, old state!
hello, new state!


True

In [14]:
machine.set_state('gas')

# 一个完整的执行顺序例子

## Callbacks

* before : 当状态转换发生时，会先调用 before 函数。发生在transition触发

* after ： 当状态转换完成后，会调用 after 函数。

* prepare: executed as soon as a transition starts, before any 'conditions' are checked or other callbacks are executed.

  * prepare will not be called unless the current state is a valid source for the named transition.

* before_state_change and 
* after_state_change : 全局的。

* prepare_event ： 全局 # 在 transitions 库中，当 send_event=True 开启时，所有回调方法都会额外接收一个 event 对象作为参数，这就导致了参数数量不匹配的问题。
* finalize_event: 执行完所有回调函数后调用，无论是否发生了状态转换。

* on_final: 当状态机进入到终止状态时，会调用 on_final 函数。callbacks which will be triggered when a state with final=True is entered.
    * self.machine = Machine(self, states=states, transitions=transitions, initial="idling", on_final="claim_success")



In [50]:
import random
from transitions import Machine, State


class Matter(object):
    heat = False
    attempts = 0

    def count_attempts(self):
        self.attempts += 1
        print("prepare: count")

    def heat_up(self):
        self.heat = random.random() < 0.5
        print(f"prepare: heat up {self.heat}")

    def before_test(self):
        print(f"before: current state: {self.state}")

    def stats(self):
        print("after: It took you %i attempts to melt the lump!" % self.attempts)
        print(f"after: current state: {self.state}")

    def make_hissing_noises(self):
        print("before_state_change: hisssss")

    def disappear(self):
        print("after_state_change: where'd all the liquid go?")

    @property
    def is_really_hot(self):
        print(f"conditions check: {self.heat}")
        return self.heat

    def raise_error(self): 
        raise ValueError("Oh no")
    def prepare_event(self): 
        print("prepare_event: I am ready!")
    def finalize(self): 
        print(f"finalize_event: Result: ")

states = ["solid", "liquid", "gas", "plasma"]

transitions = [
    {
        "trigger": "melt",
        "source": "solid",
        "dest": "liquid",
        "prepare": ["heat_up", "count_attempts"],
        "conditions": "is_really_hot",
        "before": "before_test",
        "after": "stats",
    },
]

lump = Matter()
machine = Machine(
    lump,
    states,
    transitions=transitions,
    initial="solid",
    before_state_change="make_hissing_noises",
    after_state_change="disappear",
    prepare_event='prepare_event',
    finalize_event='finalize',
    # send_event=True # 当 send_event=True 开启时，所有回调方法都会额外接收一个 event 对象作为参数，这就导致了参数数量不匹配的问题。
)
exe_ret = lump.melt()
print(f"执行结果: {exe_ret}")
# lump.melt()
# lump.melt()
# lump.melt()

prepare_event: I am ready!
prepare: heat up True
prepare: count
conditions check: True
before_state_change: hisssss
before: current state: solid
after: It took you 1 attempts to melt the lump!
after: current state: liquid
after_state_change: where'd all the liquid go?
finalize_event: Result: 
执行结果: True


In [59]:
lump.to_gas()
print()
lump.to_solid()

prepare_event: I am ready!
before_state_change: hisssss
after_state_change: where'd all the liquid go?
finalize_event: Result: 

prepare_event: I am ready!
before_state_change: hisssss
after_state_change: where'd all the liquid go?
finalize_event: Result: 


True

In [60]:
lump.melt()
lump.state

prepare_event: I am ready!
prepare: heat up False
prepare: count
conditions check: False
finalize_event: Result: 


'solid'

# Enumerations

* 

In [1]:
import enum  # Python 2.7 users need to have 'enum34' installed
from transitions import Machine

class States(enum.Enum):
    ERROR = 0
    RED = 1
    YELLOW = 2
    GREEN = 3

transitions = [['proceed', States.RED, States.YELLOW],
               ['proceed', States.YELLOW, States.GREEN],
               ['error', '*', States.ERROR]]

m = Machine(states=States, transitions=transitions, initial=States.RED)
assert m.is_RED()
assert m.state is States.RED
state = m.get_state(States.RED)  # get transitions.State object
print(state.name)  # >>> RED
m.proceed()
m.proceed()
assert m.is_GREEN()
m.error()
assert m.state is States.ERROR

RED


# transitions

## ordered transitions

## queued transitions

* 看下面例子
* prepare -> before -> on_enter_B -> on_enter_C -> after. # 无queued=True， >>> 'I am in state C now!'
>>> 'I am in state B now!' # what?
* prepare -> before -> on_enter_B -> queue(to_C) -> after  -> on_enter_C. # >>> 'I am in state B now!'
>>> 'I am in state C now!' # That's better!

* when processing events in a queue, the trigger call will always return True, since there is no way to determine at queuing time whether a transition involving queued calls will ultimately complete successfully.

In [ ]:
def go_to_C():
    global machine
    machine.to_C()

def after_advance():
    print("I am in state B now!")

def entering_C():
    print("I am in state C now!")

states = ['A', 'B', 'C']
machine = Machine(states=states, initial='A')

# we want a message when state transition to B has been completed
machine.add_transition('advance', 'A', 'B', after=after_advance)

# call transition from state B to state C
machine.on_enter_B(go_to_C)

# we also want a message when entering state C
machine.on_enter_C(entering_C)
machine.advance()
>>> 'I am in state C now!'
>>> 'I am in state B now!' # what?